## Handling Missing Values in Large-scale ML Pipelines:

**Task 1**: Impute with Mean or Median
- Step 1: Load a dataset with missing values (e.g., Boston Housing dataset).
- Step 2: Identify columns with missing values.
- Step 3: Impute missing values using the mean or median of the respective columns.

In [1]:
# write your code from here
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_boston

# Load Boston Housing dataset (deprecated in sklearn 1.2+, so alternatively create similar data)
# We'll create a DataFrame from the dataset
boston = load_boston()
df = pd.DataFrame(boston.data, columns=boston.feature_names)
df['TARGET'] = boston.target

# Introduce missing values randomly for demonstration
np.random.seed(42)
missing_rate = 0.1
n_missing_samples = int(np.floor(missing_rate * df.shape[0]))

for col in ['RM', 'AGE']:  # Pick two columns to insert missing values
    missing_indices = np.random.choice(df.index, n_missing_samples, replace=False)
    df.loc[missing_indices, col] = np.nan

print("Original data with missing values:")
print(df[['RM', 'AGE']].head(10))

# Step 2: Identify columns with missing values
missing_cols = df.columns[df.isnull().any()].tolist()
print("\nColumns with missing values:", missing_cols)

# Step 3: Impute missing values using mean or median
# Mean imputation for 'RM', Median imputation for 'AGE' (just as an example)
imputer_mean = SimpleImputer(strategy='mean')
imputer_median = SimpleImputer(strategy='median')

df['RM'] = imputer_mean.fit_transform(df[['RM']])
df['AGE'] = imputer_median.fit_transform(df[['AGE']])

print("\nData after imputation:")
print(df[['RM', 'AGE']].head(10))



ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

**Task 2**: Impute with the Most Frequent Value
- Step 1: Use the Titanic dataset and identify columns with missing values.
- Step 2: Impute categorical columns using the most frequent value.

In [ ]:
# write your code from here
import pandas as pd
import seaborn as sns
from sklearn.impute import SimpleImputer

# Step 1: Load Titanic dataset
titanic = sns.load_dataset('titanic')

# Show columns with missing values
missing_cols = titanic.columns[titanic.isnull().any()].tolist()
print("Columns with missing values:", missing_cols)

# Identify categorical columns with missing values
cat_cols = titanic.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols_with_missing = [col for col in cat_cols if col in missing_cols]

print("Categorical columns with missing values:", cat_cols_with_missing)

# Step 2: Impute categorical columns using the most frequent value (mode)
imputer = SimpleImputer(strategy='most_frequent')

# Apply imputer column-wise
titanic[cat_cols_with_missing] = imputer.fit_transform(titanic[cat_cols_with_missing])

# Verify missing values are filled
print("\nMissing values after imputation:")
print(titanic[cat_cols_with_missing].isnull().sum())


**Task 3**: Advanced Imputation - k-Nearest Neighbors
- Step 1: Implement KNN imputation using the KNNImputer from sklearn.
- Step 2: Explore how KNN imputation improves data completion over simpler methods.

In [ ]:
# write your code from here
import pandas as pd
import seaborn as sns
from sklearn.impute import KNNImputer, SimpleImputer

# Load Titanic dataset
titanic = sns.load_dataset('titanic')

# Select numerical columns for KNN imputation
num_cols = titanic.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Show missing values in numerical columns before imputation
print("Missing values in numerical columns before imputation:")
print(titanic[num_cols].isnull().sum())

# Step 1: KNN Imputation (k=3 neighbors)
knn_imputer = KNNImputer(n_neighbors=3)
titanic_knn_imputed = titanic.copy()
titanic_knn_imputed[num_cols] = knn_imputer.fit_transform(titanic[num_cols])

print("\nMissing values in numerical columns after KNN imputation:")
print(pd.DataFrame(titanic_knn_imputed[num_cols]).isnull().sum())

# Step 2: Compare with SimpleImputer (mean strategy)
simple_imputer = SimpleImputer(strategy='mean')
titanic_simple_imputed = titanic.copy()
titanic_simple_imputed[num_cols] = simple_imputer.fit_transform(titanic[num_cols])

print("\nMissing values in numerical columns after SimpleImputer (mean) imputation:")
print(pd.DataFrame(titanic_simple_imputed[num_cols]).isnull().sum())

# Optional: Show some original vs imputed values for comparison
print("\nOriginal vs KNN Imputed (first 5 rows, 'age'):")
print(pd.DataFrame({'Original': titanic['age'].head(), 'KNN Imputed': titanic_knn_imputed['age'].head()}))


## Feature Scaling & Normalization Best Practices:

**Task 1**: Standardization
- Step 1: Standardize features using StandardScaler.
- Step 2: Observe how standardization affects data distribution.

In [ ]:
# write your code from here
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Load Titanic dataset
titanic = sns.load_dataset('titanic')

# Select numerical columns
num_cols = titanic.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Drop rows with missing values in numerical columns for clean scaling
titanic_num = titanic[num_cols].dropna()

# Step 1: Standardize features using StandardScaler
scaler = StandardScaler()
titanic_scaled = scaler.fit_transform(titanic_num)

# Convert back to DataFrame for easier viewing
titanic_scaled_df = pd.DataFrame(titanic_scaled, columns=num_cols)

# Step 2: Observe effect of standardization on data distribution

# Plot original vs standardized feature distributions for 'age' and 'fare'
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Original distributions
sns.histplot(titanic_num['age'], kde=True, ax=axs[0,0], color='blue')
axs[0,0].set_title('Original Age Distribution')

sns.histplot(titanic_num['fare'], kde=True, ax=axs[0,1], color='blue')
axs[0,1].set_title('Original Fare Distribution')

# Standardized distributions
sns.histplot(titanic_scaled_df['age'], kde=True, ax=axs[1,0], color='green')
axs[1,0].set_title('Standardized Age Distribution')

sns.histplot(titanic_scaled_df['fare'], kde=True, ax=axs[1,1], color='green')
axs[1,1].set_title('Standardized Fare Distribution')

plt.tight_layout()
plt.show()


**Task 2**: Min-Max Scaling

- Step 1: Scale features to lie between 0 and 1 using MinMaxScaler.
- Step 2: Compare with standardization.

In [ ]:
# write your code from here
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load Titanic dataset
titanic = sns.load_dataset('titanic')

# Select numerical columns
num_cols = titanic.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Drop rows with missing values in numerical columns for clean scaling
titanic_num = titanic[num_cols].dropna()

# Standardization (for comparison)
scaler_standard = StandardScaler()
titanic_standardized = scaler_standard.fit_transform(titanic_num)
titanic_standardized_df = pd.DataFrame(titanic_standardized, columns=num_cols)

# Min-Max Scaling
scaler_minmax = MinMaxScaler()
titanic_minmax_scaled = scaler_minmax.fit_transform(titanic_num)
titanic_minmax_df = pd.DataFrame(titanic_minmax_scaled, columns=num_cols)

# Plotting original, standardized, and min-max scaled distributions for 'age' and 'fare'
fig, axs = plt.subplots(3, 2, figsize=(12, 12))

# Original distributions
sns.histplot(titanic_num['age'], kde=True, ax=axs[0, 0], color='blue')
axs[0, 0].set_title('Original Age Distribution')
sns.histplot(titanic_num['fare'], kde=True, ax=axs[0, 1], color='blue')
axs[0, 1].set_title('Original Fare Distribution')

# Standardized distributions
sns.histplot(titanic_standardized_df['age'], kde=True, ax=axs[1, 0], color='green')
axs[1, 0].set_title('Standardized Age Distribution')
sns.histplot(titanic_standardized_df['fare'], kde=True, ax=axs[1, 1], color='green')
axs[1, 1].set_title('Standardized Fare Distribution')

# Min-Max scaled distributions
sns.histplot(titanic_minmax_df['age'], kde=True, ax=axs[2, 0], color='red')
axs[2, 0].set_title('Min-Max Scaled Age Distribution')
sns.histplot(titanic_minmax_df['fare'], kde=True, ax=axs[2, 1], color='red')
axs[2, 1].set_title('Min-Max Scaled Fare Distribution')

plt.tight_layout()
plt.show()



**Task 3**: Robust Scaling
- Step 1: Scale features using RobustScaler, which is useful for data with outliers.
- Step 2: Assess changes in data scaling compared to other scaling methods.

In [ ]:
# write your code from here
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Load Titanic dataset
titanic = sns.load_dataset('titanic')

# Select numerical columns
num_cols = titanic.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Drop rows with missing values in numerical columns for clean scaling
titanic_num = titanic[num_cols].dropna()

# StandardScaler
scaler_standard = StandardScaler()
titanic_standardized = scaler_standard.fit_transform(titanic_num)
titanic_standardized_df = pd.DataFrame(titanic_standardized, columns=num_cols)

# MinMaxScaler
scaler_minmax = MinMaxScaler()
titanic_minmax_scaled = scaler_minmax.fit_transform(titanic_num)
titanic_minmax_df = pd.DataFrame(titanic_minmax_scaled, columns=num_cols)

# RobustScaler
scaler_robust = RobustScaler()
titanic_robust_scaled = scaler_robust.fit_transform(titanic_num)
titanic_robust_df = pd.DataFrame(titanic_robust_scaled, columns=num_cols)

# Plot distributions for 'fare' and 'age' to compare effects of scaling methods
fig, axs = plt.subplots(4, 2, figsize=(12, 14))

# Original distributions
sns.histplot(titanic_num['age'], kde=True, ax=axs[0, 0], color='blue')
axs[0, 0].set_title('Original Age Distribution')
sns.histplot(titanic_num['fare'], kde=True, ax=axs[0, 1], color='blue')
axs[0, 1].set_title('Original Fare Distribution')

# Standardized distributions
sns.histplot(titanic_standardized_df['age'], kde=True, ax=axs[1, 0], color='green')
axs[1, 0].set_title('Standardized Age Distribution')
sns.histplot(titanic_standardized_df['fare'], kde=True, ax=axs[1, 1], color='green')
axs[1, 1].set_title('Standardized Fare Distribution')

# Min-Max scaled distributions
sns.histplot(titanic_minmax_df['age'], kde=True, ax=axs[2, 0], color='red')
axs[2, 0].set_title('Min-Max Scaled Age Distribution')
sns.histplot(titanic_minmax_df['fare'], kde=True, ax=axs[2, 1], color='red')
axs[2, 1].set_title('Min-Max Scaled Fare Distribution')

# Robust scaled distributions
sns.histplot(titanic_robust_df['age'], kde=True, ax=axs[3, 0], color='purple')
axs[3, 0].set_title('Robust Scaled Age Distributi_


## Feature Selection Techniques:
### Removing Highly Correlated Features:

**Task 1**: Correlation Matrix
- Step 1: Compute correlation matrix.
- Step 2: Remove highly correlated features (correlation > 0.9).

In [ ]:

# write your code from here
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

# Load dataset (Boston Housing for example)
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

# Step 1: Compute correlation matrix
corr_matrix = df.corr().abs()

# Step 2: Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

print(f"Features to drop due to high correlation (> 0.9): {to_drop}")

# Drop highly correlated features
df_reduced = df.drop(columns=to_drop)

print("\nRemaining features after removing highly correlated ones:")
print(df_reduced.columns.tolist())


### Using Mutual Information & Variance Thresholds:

**Task 2**: Mutual Information
- Step 1: Compute mutual information between features and target.
- Step 2: Retain features with high mutual information scores.

In [ ]:
# write your code from here
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import mutual_info_classif

# Load example dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Step 1: Compute mutual information between each feature and the target
mi_scores = mutual_info_classif(X, y, discrete_features=False, random_state=42)

# Create a DataFrame of features and their MI scores
mi_df = pd.DataFrame({'Feature': X.columns, 'Mutual_Information': mi_scores})

# Sort features by MI score descending
mi_df = mi_df.sort_values(by='Mutual_Information', ascending=False)

print("Mutual Information Scores:")
print(mi_df)

# Step 2: Retain features with MI score above a threshold (e.g., > 0.01)
threshold = 0.01
selected_features = mi_df[mi_df['Mutual_Information'] > threshold]['Feature'].tolist()

print(f"\nFeatures retained (MI > {threshold}):")
print(selected_features)

# Reduced dataset with selected features
X_reduced = X[selected_features]
print(f"\nShape of reduced dataset: {X_reduced.shape}")



**Task 3**: Variance Threshold
- Step 1: Implement VarianceThreshold to remove features with low variance.
- Step 2: Analyze impact on feature space.

In [ ]:
# write your code from here
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import VarianceThreshold

# Load example dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)

print(f"Original number of features: {X.shape[1]}")

# Step 1: Remove features with low variance (threshold=0.01 here)
selector = VarianceThreshold(threshold=0.01)
X_reduced = selector.fit_transform(X)

# Features kept mask
features_kept = X.columns[selector.get_support()]

print(f"Number of features after VarianceThreshold: {X_reduced.shape[1]}")
print("Features retained:")
print(features_kept.tolist())

